In [1]:
import pandas as pd
from sklearn.model_selection import KFold
import os
from PIL import Image

# 📂 INPUT PATH (ใช้ DATA_SET_2CLASS)
base_dir = r"C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA"
input_dir = os.path.join(base_dir, "DATA_SET_2CLASS")
train_img_dir = os.path.join(input_dir, "train_images")
test_img_dir = os.path.join(input_dir, "test_images")
train_csv = os.path.join(input_dir, "train.csv")
test_csv = os.path.join(input_dir, "test.csv")

# 📂 OUTPUT PATH สำหรับ YOLOv8 (2-CLASS)
output_dir = os.path.join(input_dir, "YOLOV8")

# 📋 โหลดข้อมูล
train_image = os.listdir(train_img_dir)
train_label = pd.read_csv(train_csv)
test_image = os.listdir(test_img_dir)
test_label = pd.read_csv(test_csv)

# 🔀 ตั้งค่า K-Fold Cross Validation
kf = KFold(n_splits=4, shuffle=True, random_state=42)
train_image_df = pd.DataFrame({"filename": train_image})

# 🎯 ฟังก์ชันแปลง label เป็น binary
def label_to_binary(label: str) -> int:
    return 0 if label == "normal" else 1

# 📦 ฟังก์ชันแปลง bbox เป็น YOLO format
def convert_bbox(xmin, ymin, xmax, ymax, img_width, img_height):
    center_x = (xmin + xmax) / (2 * img_width)
    center_y = (ymin + ymax) / (2 * img_height)
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height
    return center_x, center_y, width, height

# 🔁 สร้างแต่ละ fold
for fold, (train_idx, val_idx) in enumerate(kf.split(train_image_df)):
    fold_dir = os.path.join(output_dir, f"fold_{fold+1}")

    image_train_dir = os.path.join(fold_dir, "train", "images")
    label_train_dir = os.path.join(fold_dir, "train", "labels")
    image_val_dir = os.path.join(fold_dir, "validation", "images")
    label_val_dir = os.path.join(fold_dir, "validation", "labels")
    image_test_dir = os.path.join(fold_dir, "test", "images")
    label_test_dir = os.path.join(fold_dir, "test", "labels")

    for d in [image_train_dir, label_train_dir, image_val_dir, label_val_dir, image_test_dir, label_test_dir]:
        os.makedirs(d, exist_ok=True)

    train_fold_df = train_image_df.iloc[train_idx].reset_index(drop=True)
    val_fold_df = train_image_df.iloc[val_idx].reset_index(drop=True)

    def save_images_and_labels(image_list, label_df, image_src_dir, image_dst_dir, label_dst_dir):
        for image_name in image_list:
            src_path = os.path.join(image_src_dir, image_name)
            dst_path = os.path.join(image_dst_dir, image_name)
            if os.path.exists(src_path):
                Image.open(src_path).save(dst_path)

                label_data = label_df[label_df['image_name'] == image_name]
                if not label_data.empty:
                    img = Image.open(src_path)
                    img_width, img_height = img.size
                    label_file_path = os.path.join(label_dst_dir, os.path.splitext(image_name)[0] + ".txt")
                    with open(label_file_path, 'w') as f:
                        for _, row in label_data.iterrows():
                            class_id = label_to_binary(row['label'])
                            cx, cy, w, h = convert_bbox(row['xmin'], row['ymin'], row['xmax'], row['ymax'], img_width, img_height)
                            f.write(f"{class_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

    # ✏️ Train / Val / Test
    save_images_and_labels(train_fold_df["filename"], train_label, train_img_dir, image_train_dir, label_train_dir)
    save_images_and_labels(val_fold_df["filename"], train_label, train_img_dir, image_val_dir, label_val_dir)
    save_images_and_labels(test_image, test_label, test_img_dir, image_test_dir, label_test_dir)

In [2]:
import os

def create_yaml_files(dataset_path, num_folds, class_names):
    """
    สร้างไฟล์ .yaml สำหรับแต่ละ fold เพื่อใช้เทรน YOLO

    Parameters:
        dataset_path (str): พาธหลักของ dataset
        num_folds (int): จำนวน folds ที่ต้องการสร้าง
        class_names (dict): dictionary ของ class names เช่น {0: 'normal', 1: 'abnormal'}
    """
    for fold in range(1, num_folds + 1):
        fold_dir = os.path.join(dataset_path, f"fold_{fold}").replace('\\', '/')
        yaml_path = os.path.join(fold_dir, f"fold_{fold}.yaml").replace('\\', '/')

        yaml_content = f"""# YOLO dataset config file for Fold {fold}
train: {fold_dir}/train/images
val: {fold_dir}/validation/images
test: {fold_dir}/test/images

nc: {len(class_names)}
names: [{', '.join(f'"{name}"' for name in class_names.values())}]
"""

        with open(yaml_path, "w", encoding="utf-8") as yaml_file:
            yaml_file.write(yaml_content)

        print(f"✅ สร้างไฟล์ YAML สำหรับ Fold {fold}: {yaml_path}")

# 🔧 ใช้งาน:
dataset_path = r"C:/Users/BMEi/Documents/GitHub/WORK/Windows/CODE_BME/PROJECT_MALARIA/DATA_SET_2CLASS/YOLOV8"
num_folds = 4
class_names = {
    0: "normal",
    1: "abnormal"
}

create_yaml_files(dataset_path, num_folds, class_names)


✅ สร้างไฟล์ YAML สำหรับ Fold 1: C:/Users/BMEi/Documents/GitHub/WORK/Windows/CODE_BME/PROJECT_MALARIA/DATA_SET_2CLASS/YOLOV8/fold_1/fold_1.yaml
✅ สร้างไฟล์ YAML สำหรับ Fold 2: C:/Users/BMEi/Documents/GitHub/WORK/Windows/CODE_BME/PROJECT_MALARIA/DATA_SET_2CLASS/YOLOV8/fold_2/fold_2.yaml
✅ สร้างไฟล์ YAML สำหรับ Fold 3: C:/Users/BMEi/Documents/GitHub/WORK/Windows/CODE_BME/PROJECT_MALARIA/DATA_SET_2CLASS/YOLOV8/fold_3/fold_3.yaml
✅ สร้างไฟล์ YAML สำหรับ Fold 4: C:/Users/BMEi/Documents/GitHub/WORK/Windows/CODE_BME/PROJECT_MALARIA/DATA_SET_2CLASS/YOLOV8/fold_4/fold_4.yaml


In [3]:
from ultralytics import YOLO
import shutil
import os

# 🔁 ชี้ไปยังไฟล์ YAML สำหรับ 2 คลาส
fold_paths = [
    r'C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET_2CLASS\YOLOV8\fold_1\fold_1.yaml',
    r'C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET_2CLASS\YOLOV8\fold_2\fold_2.yaml',
    r'C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET_2CLASS\YOLOV8\fold_3\fold_3.yaml',
    r'C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET_2CLASS\YOLOV8\fold_4\fold_4.yaml',
]

# 🚀 เทรน + วาลิเดต + Export ONNX
for i, path in enumerate(fold_paths, start=1):
    print(f"===== Fold {i} =====")
    model = YOLO('yolov8n.pt')

    model.train(data=path, epochs=10, device='0', project='runs_fold_2class', name=f'fold_{i}')
    model.val(device='0')

    # Export ONNX
    export_result = model.export(format='onnx')
    
    # Move exported ONNX model to custom filename
    exported_path = export_result[0] if isinstance(export_result, (list, tuple)) else export_result
    target_path = f'yolov8n_fold_{i}_2class.onnx'
    if os.path.exists(exported_path):
        shutil.move(exported_path, target_path)
        print(f"✅ Exported and saved to {target_path}")
    else:
        print(f"❌ Export failed for fold {i}")

print("✅ All folds completed.")


===== Fold 1 =====
New https://pypi.org/project/ultralytics/8.3.114 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.99  Python-3.9.13 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET_2CLASS\YOLOV8\fold_1\fold_1.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs_fold_2class, name=fold_1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=F

train: Scanning C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET_2CLASS\YOLOV8\fold_1\train\labels... 905 images, 0 backgrounds, 0 corrupt: 100%|██████████| 905/905 [00:01<00:00, 744.24it/s]


train: New cache created: C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET_2CLASS\YOLOV8\fold_1\train\labels.cache


val: Scanning C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET_2CLASS\YOLOV8\fold_1\validation\labels... 302 images, 0 backgrounds, 0 corrupt: 100%|██████████| 302/302 [00:00<00:00, 429.59it/s]

val: New cache created: C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET_2CLASS\YOLOV8\fold_1\validation\labels.cache


...


10 epochs completed in 0.029 hours.
Optimizer stripped from runs_fold_2class\fold_4\weights\last.pt, 6.2MB
Optimizer stripped from runs_fold_2class\fold_4\weights\best.pt, 6.2MB

Validating runs_fold_2class\fold_4\weights\best.pt...
Ultralytics 8.3.99  Python-3.9.13 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:05<00:00,  1.75it/s]


                   all        301      20252      0.948      0.949      0.979      0.805
                normal        301      19563      0.964      0.985      0.992      0.817
              abnormal        261        689      0.933      0.913      0.966      0.794
Speed: 0.2ms preprocess, 1.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs_fold_2class\fold_4
Ultralytics 8.3.99  Python-3.9.13 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\DATA_SET_2CLASS\YOLOV8\fold_4\validation\labels.cache... 301 images, 0 backgrounds, 0 corrupt: 100%|██████████| 301/301 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:06<00:00,  2.93it/s]


                   all        301      20252      0.949      0.948      0.979      0.805
                normal        301      19563      0.964      0.985      0.992      0.817
              abnormal        261        689      0.933      0.912      0.966      0.794
Speed: 0.4ms preprocess, 3.0ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to runs_fold_2class\fold_42
Ultralytics 8.3.99  Python-3.9.13 torch-2.6.0+cu126 CPU (Intel Core(TM) i7-14700F)

PyTorch: starting from 'runs_fold_2class\fold_4\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (5.9 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.49...
ONNX: export success  0.4s, saved as 'runs_fold_2class\fold_4\weights\best.onnx' (11.7 MB)

Export complete (0.5s)
Results saved to C:\Users\BMEi\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_MALARIA\runs_fold_2class\fold_4\weights
Predict:         yolo predict task=detect model=runs_fo

In [5]:
# 🔧 ส่วนนี้เหมือนเดิม (import และฟังก์ชัน IOU)
import os
import cv2
import logging
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
from tabulate import tabulate

logging.getLogger("ultralytics").setLevel(logging.ERROR)

def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-6)
    return iou

# 📁 พาธ
base_path = r"C:/Users/BMEi/Documents/GitHub/WORK/Windows/CODE_BME/PROJECT_MALARIA/DATA_SET_2CLASS/YOLOV8"
model_base_path = r"C:/Users/BMEi/Documents/GitHub/WORK/Windows/CODE_BME/PROJECT_MALARIA"
save_image_folder = "results_images_2class_abnormal_only"
iou_threshold = 0.5

os.makedirs(save_image_folder, exist_ok=True)

results_table = []
detailed_results = []
false_positive_images = []
fold_stats = {}

# 🔁 วน 4 folds
for i in range(1, 5):
    model_path = os.path.join(model_base_path, f"yolov8n_fold_{i}_2class.onnx")
    model = YOLO(model_path)

    images_folder = os.path.join(base_path, f"fold_{i}", "test", "images")
    labels_folder = os.path.join(base_path, f"fold_{i}", "test", "labels")

    total_gt = 0
    total_pred = 0
    correct_detect = 0
    correct_class = 0

    if i not in fold_stats:
        fold_stats[i] = {"TP": 0, "FP": 0, "FN": 0, "Images": 0}

    for filename in os.listdir(images_folder):
        if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        image_path = os.path.join(images_folder, filename)
        label_path = os.path.join(labels_folder, filename.rsplit(".", 1)[0] + ".txt")
        if not os.path.exists(label_path):
            continue

        image = cv2.imread(image_path)
        height, width = image.shape[:2]
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # ✅ Load GT
        gt_boxes = []
        with open(label_path, "r") as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                class_id, x, y, w, h = map(float, parts)
                x *= width
                y *= height
                w *= width
                h *= height
                gt_boxes.append([int(class_id), x - w/2, y - h/2, x + w/2, y + h/2])

        results = model(image_path)
        pred_boxes = [
            [int(box.cls[0].item()), *box.xyxy[0].cpu().numpy().tolist()]
            for box in results[0].boxes
        ]
        total_pred += len(pred_boxes)

        matched = set()
        pred_matched_flags = [False] * len(pred_boxes)

        for gt_class, gt_x1, gt_y1, gt_x2, gt_y2 in gt_boxes:
            total_gt += 1
            matched_pred = None
            best_iou = 0
            matched_pred_class = None

            for j, (pred_class, pred_x1, pred_y1, pred_x2, pred_y2) in enumerate(pred_boxes):
                if pred_matched_flags[j]:
                    continue
                iou = calculate_iou([gt_x1, gt_y1, gt_x2, gt_y2], [pred_x1, pred_y1, pred_x2, pred_y2])
                if iou >= iou_threshold and iou > best_iou:
                    best_iou = iou
                    matched_pred = j
                    matched_pred_class = pred_class

            if matched_pred is not None:
                correct_detect += 1
                if matched_pred_class == gt_class:
                    correct_class += 1
                pred_matched_flags[matched_pred] = True
                matched.add(matched_pred)

                pred_x1, pred_y1, pred_x2, pred_y2 = pred_boxes[matched_pred][1:]
                detailed_results.append({
                    "Fold": i, "Image": filename,
                    "GT_Class": gt_class,
                    "GT_Box": f"{gt_x1:.1f},{gt_y1:.1f},{gt_x2:.1f},{gt_y2:.1f}",
                    "Pred_Class": matched_pred_class,
                    "Pred_Box": f"{pred_x1:.1f},{pred_y1:.1f},{pred_x2:.1f},{pred_y2:.1f}",
                    "IOU": f"{best_iou:.3f}",
                    "Match": "Yes",
                    "Class_Correct": "Yes" if matched_pred_class == gt_class else "No"
                })
            else:
                detailed_results.append({
                    "Fold": i, "Image": filename,
                    "GT_Class": gt_class,
                    "GT_Box": f"{gt_x1:.1f},{gt_y1:.1f},{gt_x2:.1f},{gt_y2:.1f}",
                    "Pred_Class": None,
                    "Pred_Box": None,
                    "IOU": None,
                    "Match": "No",
                    "Class_Correct": "No"
                })

        false_positive_images.append((i, filename, image_rgb.copy(), gt_boxes, pred_boxes, matched, pred_matched_flags))

    detect_acc = correct_detect / total_gt * 100 if total_gt > 0 else 0
    class_acc = correct_class / total_gt * 100 if total_gt > 0 else 0
    results_table.append([
        f"Fold {i}",
        total_gt,
        total_pred,
        f"{detect_acc:.2f}%",
        f"{class_acc:.2f}%"
    ])

# ✅ Summary table
print("\n📊 Detection Summary:")
print(tabulate(results_table, headers=["Fold", "Total GT", "Total Prediction", "Detection Accuracy", "Classification Accuracy"], tablefmt="grid"))
pd.DataFrame(results_table, columns=["Fold", "Total GT", "Total Prediction", "Detection Accuracy", "Classification Accuracy"]).to_csv("summary_table.csv", index=False)

# ✅ Export detailed results
pd.DataFrame(detailed_results).to_csv("detection_detailed_results.csv", index=False)

# ✅ Draw only class 1
print("\n🖼️ Drawing results (only class 1)...\n")
for fold, filename, image_rgb, gt_boxes, pred_boxes, matched, pred_matched_flags in false_positive_images:
    fold_dir = os.path.join(save_image_folder, f"fold{fold}")
    os.makedirs(fold_dir, exist_ok=True)

    gt_img = image_rgb.copy()
    fn_count = 0
    for gt_class, x1, y1, x2, y2 in gt_boxes:
        if gt_class != 1: continue  # 💥 show GT only for class 1

        is_fn = True
        for j, (pred_class, px1, py1, px2, py2) in enumerate(pred_boxes):
            if pred_class != 1: continue
            if j in matched and calculate_iou([x1, y1, x2, y2], [px1, py1, px2, py2]) >= iou_threshold:
                is_fn = False
                break

        if is_fn: fn_count += 1
        color = (0, 255, 255) if not is_fn else (0, 0, 255)
        label = f"FN: {gt_class}" if is_fn else f"GT: {gt_class}"
        cv2.rectangle(gt_img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(gt_img, label, (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    pred_img = image_rgb.copy()
    tp_count, fp_count = 0, 0
    for j, (pred_class, x1, y1, x2, y2) in enumerate(pred_boxes):
        if pred_class != 1: continue  # 💥 show pred only for class 1

        if pred_matched_flags[j]:
            color = (0, 255, 255)
            label = f"TP: {pred_class}"
            tp_count += 1
        else:
            color = (255, 0, 0)
            label = f"FP: {pred_class}"
            fp_count += 1
        cv2.rectangle(pred_img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(pred_img, label, (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    fig, axs = plt.subplots(1, 2, figsize=(16, 6))
    axs[0].imshow(gt_img)
    axs[0].set_title(f"[GT: Abnormal Only] Fold {fold} - {filename}")
    axs[0].axis('off')
    axs[1].imshow(pred_img)
    axs[1].set_title(f"[Prediction] TP={tp_count}, FP={fp_count}, FN={fn_count}")
    axs[1].axis('off')
    fig.text(0.5, 0.01, "Abnormal Only — TP: match, FP: extra, FN: missed", ha='center', fontsize=10, style='italic')
    plt.tight_layout()

    save_path = os.path.join(fold_dir, f"{filename.rsplit('.', 1)[0]}_TP{tp_count}_FP{fp_count}_FN{fn_count}.jpg")
    plt.savefig(save_path)
    plt.close()

    fold_stats[fold]["TP"] += tp_count
    fold_stats[fold]["FP"] += fp_count
    fold_stats[fold]["FN"] += fn_count
    if fp_count > 0 or fn_count > 0:
        fold_stats[fold]["Images"] += 1

# ✅ Export per-fold stats
summary_rows = [
    {
        "Fold": f"Fold {k}",
        "Images": v["Images"],
        "TP": v["TP"],
        "FP": v["FP"],
        "FN": v["FN"]
    }
    for k, v in sorted(fold_stats.items())
]
pd.DataFrame(summary_rows).to_csv("summary_per_fold.csv", index=False)
print("📄 Exported per-fold summary to 'summary_per_fold.csv'")


📊 Detection Summary:
+--------+------------+--------------------+----------------------+---------------------------+
| Fold   |   Total GT |   Total Prediction | Detection Accuracy   | Classification Accuracy   |
+========+============+====================+======================+===========================+
| Fold 1 |       5922 |               6673 | 98.51%               | 95.31%                    |
+--------+------------+--------------------+----------------------+---------------------------+
| Fold 2 |       5922 |               6593 | 97.99%               | 94.19%                    |
+--------+------------+--------------------+----------------------+---------------------------+
| Fold 3 |       5922 |               6615 | 98.21%               | 94.63%                    |
+--------+------------+--------------------+----------------------+---------------------------+
| Fold 4 |       5922 |               6474 | 96.99%               | 93.28%                    |
+--------+--------